# База данных книг

В проекте сформируем представление о недавно приобретенной базе данных книг.

## Доступ к базе данных

In [1]:
# импортируем библиотеки

import pandas as pd
from sqlalchemy import create_engine

In [2]:
# установим параметры

db_config = {'user': '', # имя пользователя
'pwd': '', # пароль
'host': '',
'port': 6432, # порт подключения
'db': ''} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраним коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
# создадим функцию для запросов к базе

def select(query):
    return pd.io.sql.read_sql(query, con = engine) 

## Знакомство с данными

Выведем по 5 первых строк каждой таблицы.

In [4]:
for table in ['books', 'authors', 'ratings', 'reviews', 'publishers']:
    print('Table: {}'.format(table))
    display(select('Select * from {} limit(5)'.format(table)))
    print('\n\n')

Table: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268





Table: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd





Table: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2





Table: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...





Table: publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Количество книг, вышедших после 1 января 2000 года

In [5]:
select('''SELECT COUNT(DISTINCT book_id) 
        FROM books
        WHERE CAST(publication_date AS date) > '2000-01-01'
       ''')

,count
0,819


- 819 книг вышло после 1 января 2000 года.

## Количество обзоров и средняя оценка для каждой книги

In [6]:
select('''SELECT b.title, COUNT(DISTINCT re.review_id) as reviews, AVG(ra.rating) as avg_rating
        FROM books b
        LEFT JOIN reviews re on b.book_id=re.book_id
        LEFT JOIN ratings ra on b.book_id=ra.book_id
        GROUP BY (b.title)
       ''')

,title,reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
994,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
995,Xenocide (Ender's Saga #3),3,3.400000
996,Year of Wonders,4,3.200000
997,You Suck (A Love Story #2),2,4.500000


In [7]:
select('''SELECT b.title, COUNT(DISTINCT re.review_id) as reviews, AVG(ra.rating) as avg_rating, 
        COUNT(DISTINCT ra.rating_id) as rating_cnt
        FROM books b
        LEFT JOIN reviews re on b.book_id=re.book_id
        LEFT JOIN ratings ra on b.book_id=ra.book_id
        WHERE b.num_pages > 50
        GROUP BY (b.title)
        HAVING AVG(ra.rating) = (SELECT MAX(rating) from ratings) 
        AND COUNT(DISTINCT ra.rating_id) > 2 
        LIMIT 20
       ''')

,title,reviews,avg_rating,rating_cnt
0,A Dirty Job (Grim Reaper #1),4,5.0,4
1,In the Hand of the Goddess (Song of the Liones...,2,5.0,3
2,Moneyball: The Art of Winning an Unfair Game,3,5.0,3
3,School's Out—Forever (Maximum Ride #2),3,5.0,4


- Только четыре книги (толще 50 страниц) имеют максимальный средний рейтинг при условии, что он был расчитан по более чем двум оценкам;
- Также на эти книги есть от 2 до 4 обзоров.

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [8]:
select('''SELECT p.publisher, count(DISTINCT b.book_id) AS books_cnt
        FROM books b
        JOIN publishers p on b.publisher_id=p.publisher_id
        WHERE b.book_id IN 
        (SELECT book_id
        FROM books
        WHERE num_pages > 50)
        GROUP BY (p.publisher)
        ORDER BY books_cnt DESC
        LIMIT 5
        ''')

,publisher,books_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


- Penguin Books - издательство-лидер по количеству выпущенных книг толще 50 страниц (не брошюр).

## Автор с самой высокой средней оценкой книг, которые имеют 50 и более оценок

In [9]:
select('''                           
        SELECT a.author, AVG(ra.rating) AS avg_rating
        FROM books b
        JOIN authors a on b.author_id=a.author_id
        JOIN ratings ra on b.book_id=ra.book_id
        WHERE b.book_id IN (SELECT book_id
                           FROM ratings
                           GROUP BY (book_id)
                           HAVING COUNT(DISTINCT rating_id) > 50)
        GROUP BY a.author
        ORDER BY avg_rating DESC 
        LIMIT 5

''')

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


- J.K. Rowling/Mary GrandPré - автор с наибольшим средним рейтингом книг, которые были оценены 50 и более раз;
- Не сильно по этому показателю от лидера отстают авторы Markus Zusak/Cao Xuân Việt Khương и J.R.R. Tolkien.

## Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [10]:
select('''
        WITH users AS (SELECT username
                       FROM ratings
                       GROUP BY username
                       HAVING COUNT(DISTINCT rating_id) > 50),
                       
        reviews_per_user AS (SELECT COUNT(DISTINCT review_id) reviews
                             FROM reviews
                             WHERE username in (SELECT * FROM users)
                             GROUP BY username)
        
        SELECT ROUND(AVG(reviews), 2) FROM reviews_per_user
        
''')

,round
0,24.33


- Каждый пользователь, который оставил больше 50 оценок, в среднем писал около 24 обзоров.

## Выводы

- После 1 января 2000 года вышло 819 книг;
- Только четыре книги (толще 50 страниц) имеют максимальный средний рейтинг при условии, что он был расчитан по более чем двум оценкам. Также на эти книги есть от 2 до 4 обзоров;
- Penguin Books - издательство-лидер по количеству выпущенных книг (брошюры исключены);
- J.K. Rowling/Mary GrandPré - автор с наибольшим средним рейтингом книг, которые были оценены 50 и более раз. Не сильно по этому показателю от лидера отстают авторы Markus Zusak/Cao Xuân Việt Khương и J.R.R. Tolkien;
- Каждый пользователь, который оставил больше 50 оценок, в среднем писал около 24 обзоров.